### Importing shit

In [66]:
import cv2
import numpy as np
import time

### Loading pic

In [104]:
image = cv2.imread("test3.jpg")
cv2.imshow("aaaa",image)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [116]:
def fn(x):
    sigma = 2
    return 1/np.sqrt(2*3.1416*sigma**2)*np.exp(-(x**2)/(2*sigma**2))
def convolve(arr1, arr2):
    #print(arr1, arr2)
    res = sum(sum(arr1*arr2)) # Хуйни проверку на одинаковый shape
    #print(res)
    #time.sleep(1)
    return int(res)
def blur(side, arr):
    diap = np.linspace(-(side - 1) / 2, (side - 1) / 2, side)
    d1kern = np.array(list(map(fn, diap)))
    print(arr.shape)
    newarr = np.zeros(arr.shape, dtype = "uint8")
    print(newarr.shape)
    kern = np.outer(d1kern, d1kern) # np.array(np.fromfunction(fn, (-kside,kside)))
    arr = np.vstack([np.zeros((int(side/2),arr.shape[0])), arr.T, np.zeros((int(side/2),arr.shape[0]))])
    arr =  np.vstack([np.zeros((int(side/2),arr.shape[0])), arr.T, np.zeros((int(side/2),arr.shape[0]))])
    print(kern)
    arr = arr
    for y in range(int(side/2), arr.shape[1]-int(side/2)):
        for x in range(int(side/2), arr.shape[0]-int(side/2)):
            sub = arr[x-int(side/2):(x+int(side/2)+1), y-int(side/2):(y+int(side/2)+1)]
            #print(sub.shape)
            #time.sleep(1)
            newarr[x-int(side/2),y-int(side/2)] = convolve(sub, kern)
            #print(newarr[x-int(side/2),y-int(side/2)])
            #print(x,y)
            #print(sub.shape)
    return newarr
    #np.vstack([np.zeros((1,6)), np.vstack([np.zeros((1,arr.shape[0])), arr.T, np.zeros((1,arr.shape[0]))]).T, np.zeros((1,6))])
#blur(5, image[][][0])
def blurRGB(image, side):
    (b, g, r) = cv2.split(image)
    #print(b,g,r,sep="\n")
    b = blur(side, b)
    g = blur(side, g)
    r = blur(side, r)
    #print(b.shape)
    merged = cv2.merge([b, g, r])
    #print(b,g,r,sep="\n")
    cv2.imshow("Merged", merged)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    cv2.imwrite("test"+str(side)+".jpg", merged)
blurRGB(image, 11)

(257, 386)
(257, 386)
[[7.68101499e-05 2.36591918e-04 5.67554507e-04 1.06033141e-03
  1.54277310e-03 1.74819095e-03 1.54277310e-03 1.06033141e-03
  5.67554507e-04 2.36591918e-04 7.68101499e-05]
 [2.36591918e-04 7.28754412e-04 1.74819095e-03 3.26605068e-03
  4.75207570e-03 5.38480723e-03 4.75207570e-03 3.26605068e-03
  1.74819095e-03 7.28754412e-04 2.36591918e-04]
 [5.67554507e-04 1.74819095e-03 4.19369209e-03 7.83484829e-03
  1.13996370e-02 1.29174810e-02 1.13996370e-02 7.83484829e-03
  4.19369209e-03 1.74819095e-03 5.67554507e-04]
 [1.06033141e-03 3.26605068e-03 7.83484829e-03 1.46374237e-02
  2.12973257e-02 2.41330317e-02 2.12973257e-02 1.46374237e-02
  7.83484829e-03 3.26605068e-03 1.06033141e-03]
 [1.54277310e-03 4.75207570e-03 1.13996370e-02 2.12973257e-02
  3.09874261e-02 3.51133540e-02 3.09874261e-02 2.12973257e-02
  1.13996370e-02 4.75207570e-03 1.54277310e-03]
 [1.74819095e-03 5.38480723e-03 1.29174810e-02 2.41330317e-02
  3.51133540e-02 3.97886427e-02 3.51133540e-02 2.4133031